In [ ]:
# Demo

# -*- coding: utf-8 -*-
"""
Created on Sat Apr 30 22:40:48 2022

@author: DELL
"""
#%% code interpretation
# 1.random forest test
# 2.Reference 
#   https://data36.com/random-forest-in-python/
#   https://mljar.com/blog/feature-importance-in-random-forest/#:~:text=Random%20Forest%20Built%2Din%20Feature,a%20set%20of%20Decision%20Trees.
#%%
import pandas as pd
import shap
from matplotlib import pyplot as plt
import numpy as np
import xarray as xr
import glob

In [ ]:
# demo
#### import dataset
df = pd.read_csv("/mnt/d/cym/RF_data/possum.csv")
df.sample(5, random_state=44)
df = df.dropna()

X = df.drop(["case", "site", "Pop", "sex"], axis=1)
y = df["sex"]

df

#### training random forest model
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import export_graphviz
import pydotplus
from IPython.display import Image

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=44)

rf_model = RandomForestClassifier(n_estimators=100, max_features="auto", bootstrap = True, random_state=44,
                                  oob_score=True)
rf_model.fit(X_train, y_train)

#### predict with our model 
predictions = rf_model.predict(X_test)
predictions

rf_model.predict_proba(X_test)
rf_model.classes_
importances = rf_model.feature_importances_
rf_model.predict_proba(X_test)

######################################################################
# ## Visiable is <one> decision tree
# estimator = rf_model.estimators_[5]
# dot_data = export_graphviz(estimator, out_file=None, 
#                 feature_names=rf_model.feature_names_in_,
#                 class_names=rf_model.classes_,
#                 rounded = True, proportion = False, 
#                 precision = 2, filled = True)
# graph = pydotplus.graph_from_dot_data(dot_data)
# # 使用ipython的终端jupyter notebook显示。
# Image(graph.create_png())
######################################################################

#### graph fig1
plt.rcParams.update({'figure.figsize': (12.0, 8.0)})
plt.rcParams.update({'font.size': 14})
plt.barh(X.columns, importances)

## fig2
sorted_idx = importances.argsort()
plt.barh(X.columns[sorted_idx], importances[sorted_idx])
plt.xlabel("Random Forest Feature Importance")
rf_model.oob_score_


In [ ]:
# %run identify_staionpointis.ipynb
# from ipynb.fs.defs.identify_staionpointis import identify_staionpoint_isin_whichgird
from ipynb.fs.full.identify_staionpointis import identify_staionpoint_isin_whichgird

# identify_staionpoint_isin_whichgird(lon,lat,loc)
# identify_staionpoint_isin_whichgird(Lon,Lat,loc,'Wheeler')

#  case one WRR paper

In [ ]:
# =============================================================================
# Variabels:TWC,ET,Pre,PET,SM_pre,NDVI
# =============================================================================
## load data
drought_event = pd.read_csv("/mnt/d/cym/RF_data/2021wr031829-sup-0002-data set si-s01.csv")

grace = xr.open_dataset("/mnt/d/cym/RF_data/7670849/01_monthly_grids_ensemble_means_allmodels/01_monthly_grids_ensemble_means_allmodels/GRACE_REC_v03_JPL_ERA5_monthly_ensemble_mean.nc").rec_ensemble_mean
with xr.open_mfdataset('/mnt/d/cym/RF_data/Evapotranspiration-DOLCE/*.nc') as f:  # 批量读取文件并合并
    ET = f['hfls']
# EP = xr.open_dataset("/mnt/e/Research_life/DATA/GLEAM/data/v3.6a/monthly/Ep_1980-2021_GLEAM_v3.6a_MO.nc").Ep
EP =xr.open_dataset("/mnt/d/cym/RF_data/cru_ts4.02.1901.2017.pet.dat.nc",engine='netcdf4').pet

Pre = xr.open_dataset("/mnt/d/cym/RF_data/Precipitation/precip.mon.total.v2018.nc").precip
# SM_Surf = xr.open_dataset("/mnt/e/Research_life/DATA/GLEAM/data/v3.6a/monthly/SMsurf_1980-2021_GLEAM_v3.6a_MO.nc").SMsurf
SM_Surf = xr.open_dataset('/mnt/d/cym/RF_data/sm.nc').sm

NDVI = xr.open_dataset("/mnt/d/cym/RF_data/NDVI_mvc_USA_mon_1982_2015.nc").NDVI_mvc

Pre['lon'] = Pre['lon'].values-360 # original data ranges from 0 to 360

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

## interp
Lat = np.arange(19.5,50.5,0.5)
Lon=np.arange(-110.5,-89.5,0.5)

grace = grace.interp(
    lat = Lat,
    lon = Lon,
    method_non_numeric='nearest',
    kwargs={
        "fill_value":"extrapolate"
        }
    )

ET = ET.interp(
    lat = Lat,
    lon = Lon,
    method_non_numeric='nearest',
    kwargs={
        "fill_value":"extrapolate"
        }
    )

EP = EP.interp(
    lat = Lat,
    lon = Lon,
    method_non_numeric='nearest',
    kwargs={
        "fill_value":"extrapolate"
        }
    )

Pre = Pre.interp(
    lat = Lat,
    lon = Lon,
    method_non_numeric='nearest',
    kwargs={
        "fill_value":"extrapolate"
        }
    )

SM_Surf = SM_Surf.interp(
    lat = Lat,
    lon = Lon,
    method_non_numeric='nearest',
    kwargs={
        "fill_value":"extrapolate"
        }
    )

NDVI = NDVI.interp(
    lat = Lat,
    lon = Lon,
    method_non_numeric='nearest',
    kwargs={
        "fill_value":"extrapolate"
        }
    )

# NDVI['time'] = np.arange('2001-01', '2016-01', dtype='datetime64[M]')


In [ ]:
## standardize data and uniform to same time scale 1982-2015
grace = ((grace-grace.mean(axis=0))/grace.std(axis=0,ddof=1)).loc['1982':'2015',::]
ET = ((ET-ET.mean(axis=0))/ET.std(axis=0,ddof=1)).loc['1982':'2015',::]
EP = ((EP-EP.mean(axis=0))/EP.std(axis=0,ddof=1)).loc['1982':'2015',::]
NDVI = ((NDVI-NDVI.mean(axis=0))/NDVI.std(axis=0,ddof=1)).loc['1982':'2015',::]
SM_Surf = ((SM_Surf-SM_Surf.mean(axis=0))/SM_Surf.std(axis=0,ddof=1)).loc['1982':'2015',::]
Pre = ((Pre-Pre.mean(axis=0))/Pre.std(axis=0,ddof=1)).loc['1982':'2015',::]
drought_event = drought_event[(drought_event['year']>1982)&(drought_event['year']<2016)]

In [ ]:
drought_event

## site one(wheel)

In [ ]:
## load station site location 
loc = pd.read_excel('/mnt/d/cym/RF_data/wrr_site_loc.xlsx').dropna(how='any')
lat,lon = identify_staionpoint_isin_whichgird(Lon,Lat,loc,'Wheeler')[0]

time = pd.to_datetime(drought_event[['year']+['month']+['day']])
drought_num = drought_event.iloc[:,4]

# bool_value[j] = i.time.dt.strftime('%Y-%m').isin(time.dt.strftime('%Y-%m'))

et = ET.loc[ET.time.dt.strftime('%Y-%m').isin(time.dt.strftime('%Y-%m')),lat,lon]
Et = np.squeeze(et.values.reshape(-1,1))

twc = grace.loc[grace.time.dt.strftime('%Y-%m').isin(time.dt.strftime('%Y-%m')),lat,lon]
TWC =  np.squeeze(twc.values.reshape(-1,1))

Ep = np.squeeze(EP.loc[EP.time.dt.strftime('%Y-%m').isin(time.dt.strftime('%Y-%m')),lat,lon].values.reshape(-1,1))
pre = np.squeeze(Pre.loc[Pre.time.dt.strftime('%Y-%m').isin(time.dt.strftime('%Y-%m')),lat,lon].values.reshape(-1,1))
SM_surf = np.squeeze(SM_Surf.loc[SM_Surf.time.dt.strftime('%Y-%m').isin((time-np.timedelta64(1,'M')).dt.strftime('%Y-%m')),lat,lon].values.reshape(-1,1))
ndvi = np.squeeze(NDVI.loc[NDVI.time.dt.strftime('%Y-%m').isin(time.dt.strftime('%Y-%m')),lat,lon].values.reshape(-1,1))

dro_inp = pd.concat([drought_num,pd.Series(Et),pd.Series(TWC),pd.Series(Ep),pd.Series(pre),pd.Series(SM_surf),pd.Series(ndvi)],axis=1)
dro_inp = dro_inp.dropna(how='any')
dro_inp.columns = ['Binary','Et','TWC','Ep','pre','SM_surf','ndvi']

### random forest model

In [ ]:
#### random forest model
X = dro_inp.iloc[:,1:]
y = dro_inp.iloc[:,0]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=11)

rf_model = RandomForestClassifier(n_estimators=500, max_features="auto", bootstrap = True, random_state=44,oob_score=True)
rf_model.fit(X_train, y_train)

## predict with our model 
predictions = rf_model.predict(X_test)
predictions

rf_model.predict_proba(X_test)
rf_model.classes_
importances = rf_model.feature_importances_

## graph fig1
plt.rcParams.update({'figure.figsize': (12.0, 8.0)})
plt.rcParams.update({'font.size': 14})
plt.barh(X.columns, importances)

## fig2
sorted_idx = importances.argsort()
plt.barh(X.columns[sorted_idx], importances[sorted_idx])
plt.xlabel("Random Forest Feature Importance")

## report
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
result = confusion_matrix(y_test, predictions)
print("Confusion Matrix:")
print(result)
result1 = classification_report(y_test, predictions)
print("Classification Report:")
print (result1)
result2 = accuracy_score(y_test,predictions)
print("Accuracy:",result2)
print("oob error:",1-rf_model.oob_score_)

## site two(Potter)

In [ ]:
## load station site location 
lat,lon = identify_staionpoint_isin_whichgird(Lon,Lat,loc,'Potter')[0]

time = pd.to_datetime(drought_event[['year']+['month']+['day']])
drought_num = drought_event.iloc[:,4]

# bool_value[j] = i.time.dt.strftime('%Y-%m').isin(time.dt.strftime('%Y-%m'))

et = ET.loc[ET.time.dt.strftime('%Y-%m').isin(time.dt.strftime('%Y-%m')),lat,lon]
Et = np.squeeze(et.values.reshape(-1,1))

twc = grace.loc[grace.time.dt.strftime('%Y-%m').isin(time.dt.strftime('%Y-%m')),lat,lon]
TWC =  np.squeeze(twc.values.reshape(-1,1))

Ep = np.squeeze(EP.loc[EP.time.dt.strftime('%Y-%m').isin(time.dt.strftime('%Y-%m')),lat,lon].values.reshape(-1,1))
pre = np.squeeze(Pre.loc[Pre.time.dt.strftime('%Y-%m').isin(time.dt.strftime('%Y-%m')),lat,lon].values.reshape(-1,1))
SM_surf = np.squeeze(SM_Surf.loc[SM_Surf.time.dt.strftime('%Y-%m').isin((time-np.timedelta64(1,'M')).dt.strftime('%Y-%m')),lat,lon].values.reshape(-1,1))
ndvi = np.squeeze(NDVI.loc[NDVI.time.dt.strftime('%Y-%m').isin(time.dt.strftime('%Y-%m')),lat,lon].values.reshape(-1,1))

dro_inp = pd.concat([drought_num,pd.Series(Et),pd.Series(TWC),pd.Series(Ep),pd.Series(pre),pd.Series(SM_surf),pd.Series(ndvi)],axis=1)
dro_inp = dro_inp.dropna(how='any')
dro_inp.columns = ['Binary','Et','TWC','Ep','pre','SM_surf','ndvi']

In [ ]:
print(dro_inp.shape)
dro_inp

### random forest model 

In [ ]:
#### random forest model
X = dro_inp.iloc[:,1:]
y = dro_inp.iloc[:,0]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=44)

rf_model = RandomForestClassifier(n_estimators=500, max_features="auto", bootstrap = True, random_state=44,oob_score=True)
rf_model.fit(X_train, y_train)

## predict with our model 
predictions = rf_model.predict(X_test)
predictions

rf_model.predict_proba(X_test)
rf_model.classes_
importances = rf_model.feature_importances_

## graph fig1
plt.rcParams.update({'figure.figsize': (12.0, 8.0)})
plt.rcParams.update({'font.size': 14})
plt.barh(X.columns, importances)

## fig2
sorted_idx = importances.argsort()
plt.barh(X.columns[sorted_idx], importances[sorted_idx])
plt.xlabel("Random Forest Feature Importance")

## report
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
result = confusion_matrix(y_test, predictions)
print("Confusion Matrix:")
print(result)
result1 = classification_report(y_test, predictions)
print("Classification Report:",)
print (result1)
result2 = accuracy_score(y_test,predictions)
print("Accuracy:",result2)
print("oob error:",1-rf_model.oob_score_)

## site three(Hudspeth)

In [ ]:
## load station site location 
lat,lon = identify_staionpoint_isin_whichgird(Lon,Lat,loc,'Hudspeth')[0]

time = pd.to_datetime(drought_event[['year']+['month']+['day']])
drought_num = drought_event.iloc[:,3]

# bool_value[j] = i.time.dt.strftime('%Y-%m').isin(time.dt.strftime('%Y-%m'))

et = ET.loc[ET.time.dt.strftime('%Y-%m').isin(time.dt.strftime('%Y-%m')),lat,lon]
Et = np.squeeze(et.values.reshape(-1,1))

twc = grace.loc[grace.time.dt.strftime('%Y-%m').isin(time.dt.strftime('%Y-%m')),lat,lon]
TWC =  np.squeeze(twc.values.reshape(-1,1))

Ep = np.squeeze(EP.loc[EP.time.dt.strftime('%Y-%m').isin(time.dt.strftime('%Y-%m')),lat,lon].values.reshape(-1,1))
pre = np.squeeze(Pre.loc[Pre.time.dt.strftime('%Y-%m').isin(time.dt.strftime('%Y-%m')),lat,lon].values.reshape(-1,1))
SM_surf = np.squeeze(SM_Surf.loc[SM_Surf.time.dt.strftime('%Y-%m').isin((time-np.timedelta64(1,'M')).dt.strftime('%Y-%m')),lat,lon].values.reshape(-1,1))
ndvi = np.squeeze(NDVI.loc[NDVI.time.dt.strftime('%Y-%m').isin(time.dt.strftime('%Y-%m')),lat,lon].values.reshape(-1,1))

dro_inp = pd.concat([drought_num,pd.Series(Et),pd.Series(TWC),pd.Series(Ep),pd.Series(pre),pd.Series(SM_surf),pd.Series(ndvi)],axis=1)
dro_inp = dro_inp.dropna(how='any')
dro_inp.columns = ['Binary','Et','TWC','Ep','pre','SM_surf','ndvi']

### random forest model

In [ ]:
#### random forest model
X = dro_inp.iloc[:,1:]
y = dro_inp.iloc[:,0]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=44)

rf_model = RandomForestClassifier(n_estimators=500, max_features="auto", bootstrap = True, random_state=44,oob_score=True)
rf_model.fit(X_train, y_train)

## predict with our model 
predictions = rf_model.predict(X_test)
predictions

rf_model.predict_proba(X_test)
rf_model.classes_
importances = rf_model.feature_importances_

## graph fig1
plt.rcParams.update({'figure.figsize': (12.0, 8.0)})
plt.rcParams.update({'font.size': 14})
plt.barh(X.columns, importances)

## fig2
sorted_idx = importances.argsort()
plt.barh(X.columns[sorted_idx], importances[sorted_idx])
plt.xlabel("Random Forest Feature Importance")

## report
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
result = confusion_matrix(y_test, predictions)
print("Confusion Matrix:")
print(result)
result1 = classification_report(y_test, predictions)
print("Classification Report:",)
print (result1)
result2 = accuracy_score(y_test,predictions)
print("Accuracy:",result2)
print("oob error:",1-rf_model.oob_score_)

In [ ]:
time = pd.to_datetime(drought_event[['year']+['month']+['day']])
time

# Create Fuction: For Total site agg for function

In [ ]:
## Extract all variable of the point according to <one> location of the selected point <site_name>
def total_RF(Lon,Lat,loc,site_name,num):
    ## load station site location 
    lat,lon = identify_staionpoint_isin_whichgird(Lon,Lat,loc,site_name)[0]
    # drought_event = pd.read_csv("/mnt/c/Users/DELL/Desktop/2021wr031829-sup-0002-data set si-s01.csv")
    # drought_event = drought_event[(drought_event['year']>1982)&(drought_event['year']<2016)]
    time = pd.to_datetime(drought_event[['year']+['month']+['day']])
    drought_num = (drought_event.iloc[:,num+3]).reset_index(drop=True)
    # .reset_index(drop=True, inplace=True)

    # bool_value[j] = i.time.dt.strftime('%Y-%m').isin(time.dt.strftime('%Y-%m'))

    et = ET.loc[ET.time.dt.strftime('%Y-%m').isin(time.dt.strftime('%Y-%m')),lat,lon]
    Et = np.squeeze(et.values.reshape(-1,1))

    twc = grace.loc[grace.time.dt.strftime('%Y-%m').isin(time.dt.strftime('%Y-%m')),lat,lon]
    TWC =  np.squeeze(twc.values.reshape(-1,1))

    Ep = np.squeeze(EP.loc[EP.time.dt.strftime('%Y-%m').isin(time.dt.strftime('%Y-%m')),lat,lon].values.reshape(-1,1))
    pre = np.squeeze(Pre.loc[Pre.time.dt.strftime('%Y-%m').isin(time.dt.strftime('%Y-%m')),lat,lon].values.reshape(-1,1))
    SM_surf = np.squeeze(SM_Surf.loc[SM_Surf.time.dt.strftime('%Y-%m').isin((time-np.timedelta64(1,'M')).dt.strftime('%Y-%m')),lat,lon].values.reshape(-1,1))
    ndvi = np.squeeze(NDVI.loc[NDVI.time.dt.strftime('%Y-%m').isin(time.dt.strftime('%Y-%m')),lat,lon].values.reshape(-1,1))

    dro_inp = pd.concat([drought_num,pd.Series(Et),pd.Series(TWC),pd.Series(Ep),pd.Series(pre),pd.Series(SM_surf),pd.Series(ndvi)],axis=1)
    dro_inp = dro_inp.dropna(how='any')
    dro_inp.columns = ['Binary','Et','TWC','Ep','pre','SM_surf','ndvi']
    return dro_inp,lat,lon

if __name__=='__main__':
    Lat = np.arange(19.5,50.5,0.5)
    Lon=np.arange(-110.5,-89.5,0.5)
    loc = pd.read_excel('/mnt/d/cym/RF_data/wrr_site_loc.xlsx').dropna(how='any')
    num = 3
    site_name = loc['name'][num]
    a=total_RF(Lon,Lat,loc,site_name,num)[0]

## calculate all site station predictors variables

In [ ]:
## Extract all variable of the point according to <all of> location of the selected point
Lat = np.arange(19.5,50.5,0.5)
Lon=np.arange(-110.5,-89.5,0.5)
loc = pd.read_excel('/mnt/c/Users/DELL/Desktop/wrr_site_loc.xlsx').dropna(how='any')

dro_inp = pd.DataFrame(columns = ['Binary','Et','TWC','Ep','pre','SM_surf','ndvi'])
LAT = []
LON = []
for i in range(loc.shape[0]):
    dro_inp = pd.concat([dro_inp,total_RF(Lon,Lat,loc,loc['name'][i],i)[0]]) # [0] for ['Binary','Et','TWC','Ep','pre','SM_surf','ndvi']
    LON.append(total_RF(Lon,Lat,loc,loc['name'][i],i)[1]) # [1] for lat
    LAT.append(total_RF(Lon,Lat,loc,loc['name'][i],i)[2]) # [2] for lon
    # lat = np.concatenate(total_RF(Lon,Lat,loc,loc['name'][i])[1])
    # lon = np.concatenate(total_RF(Lon,Lat,loc,loc['name'][i])[2])
# total_RF(Lon,Lat,loc,loc['name'][0])


In [ ]:
dro_inp
# LAT

# lat,lon = identify_staionpoint_isin_whichgird(Lon,Lat,loc,loc['name'][2])[]

# identify_staionpoint_isin_whichgird(Lon,Lat,loc,loc['name'][2])
# total_RF(Lon,Lat,loc,loc['name'][i])[1]


In [ ]:
# dro_inp
Lat = np.arange(19.5,50.5,0.5)
Lon=np.arange(-110.5,-89.5,0.5)
loc = pd.read_excel('/mnt/c/Users/DELL/Desktop/wrr_site_loc.xlsx').dropna(how='any')
total_RF(Lon,Lat,loc,loc['name'][0])

## RF model for all site stations 

In [ ]:
#### random forest model
X = dro_inp.iloc[:,1:]
y = dro_inp.iloc[:,0]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=111)

rf_model = RandomForestClassifier(n_estimators=500, max_features="auto", bootstrap = True, random_state=111,oob_score=True)
rf_model.fit(X_train, y_train)

## predict with our model 
predictions = rf_model.predict(X_test)
predictions

rf_model.predict_proba(X_test)
rf_model.classes_
importances = rf_model.feature_importances_

## graph fig1
plt.rcParams.update({'figure.figsize': (12.0, 8.0)})
plt.rcParams.update({'font.size': 14})
plt.barh(X.columns, importances)

## fig2
sorted_idx = importances.argsort()
plt.barh(X.columns[sorted_idx], importances[sorted_idx])
plt.xlabel("Random Forest Feature Importance")

## report
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
result = confusion_matrix(y_test, predictions)
print("Confusion Matrix:")
print(result)
result1 = classification_report(y_test, predictions)
print("Classification Report:")
print (result1)
result2 = accuracy_score(y_test,predictions)
print("Accuracy:",result2)
print("oob error:",1-rf_model.oob_score_)

In [ ]:
from sklearn.tree import export_graphviz
import pydotplus
from IPython.display import Image

######################################################################
## Visiable is <one> decision tree
estimator = rf_model.estimators_[10]
dot_data = export_graphviz(estimator, out_file=None, 
                feature_names=rf_model.feature_names_in_,
                class_names=['0','1'],
                rounded = True, proportion = False, 
                precision = 2, filled = True)
graph = pydotplus.graph_from_dot_data(dot_data)
# 使用ipython的终端jupyter notebook显示。
Image(graph.create_png())
######################################################################


In [ ]:
rf_model.predict_proba(X_test)[:,1] # drought probability
del X,X_test,X_train,y,y_test,y_train

## Predict drought probability by using trained RF model

### multi processes vocality is still slow, duo to two <for> commands always extract netccdf document each time. 
### Therefore, we can read .nc document in advance, and then exact two <for> commands


In [ ]:
# multi processes vocality is still slow, duo to two <for> commands always extract netccdf document each time. 
# Therefore, we can read .nc document in advance, and then exact two <for> commands

## extract six variables for all of grid

# Variabels:TWC,ET,Pre,PET,SM_Surf,NDVI
start_yr = 1982
end_yr = 2015

# start_yr = 2010
# end_yr = 2012

interval = end_yr-start_yr+1
n_var = 6
x_predict = np.full((12*interval*n_var,len(grace.lat),len(grace.lon)),np.nan)

grace_mat = grace.loc[str(start_yr):str(end_yr)].values
ET_mat = ET.loc[str(start_yr):str(end_yr)].values
EP_mat = EP.loc[str(start_yr):str(end_yr)].values
Pre_mat = Pre.loc[str(start_yr):str(end_yr)].values
SM_Surf_mat = SM_Surf.loc[str(start_yr):str(end_yr)].values
NDVI_mat = NDVI.loc[str(start_yr):str(end_yr)].values

for i in range(len(grace.lat)):
    for j in range(len(grace.lon)):
        x_predict[:,i,j] = np.concatenate((grace_mat[:,i,j],ET_mat[:,i,j],EP_mat[:,i,j],
                                            Pre_mat[:,i,j],SM_Surf_mat[:,i,j],NDVI_mat[:,i,j]),axis=0)


# ## multi processes
# import time
# from multiprocessing.dummy import Pool as ThreadPool
# import threading
# start_yr = 2010
# end_yr = 2012
# interval = end_yr-start_yr+1
# n_var = 6
# x_predict = np.full((12*interval*n_var,len(grace.lat),len(grace.lon)),np.nan)

# def predict(grace,ET,x_predict,start_yr,end_yr):
#     for i in range(len(grace.lat)):
#         for j in range(len(grace.lon)):
#             x_predict[:,i,j] = np.concatenate((grace.loc[str(start_yr):str(end_yr)][:,i,j],ET.loc[str(start_yr):str(end_yr)][:,i,j],
#                                 EP.loc[str(start_yr):str(end_yr)][:,i,j],Pre.loc[str(start_yr):str(end_yr)][:,i,j],
#                                 SM_Surf.loc[str(start_yr):str(end_yr)][:,i,j],NDVI.loc[str(start_yr):str(end_yr)][:,i,j]),axis=0)
#     return x_predict

# t = threading.Thread(target=predict,args=(grace,ET,x_predict,start_yr,end_yr))
# t.start()
# t.join()

In [ ]:
## save x_predict dataset
import scipy.io as io
io.savemat('/mnt/e/Research_life/DATA/RF_model/x_predict.mat', {'name': x_predict})

In [ ]:
# import time
# from multiprocessing.dummy import Pool as ThreadPool

# def predict(grace,ET,x_predict,start_yr,end_yr):
#     for i in range(len(grace.lat)):
#         for j in range(len(grace.lon)):
#             x_predict[:,i,j] = np.concatenate((grace.loc[str(start_yr):str(end_yr)][:,i,j],ET.loc[str(start_yr):str(end_yr)][:,i,j]),axis=0)
#     return x_predict



# def process(grace):
#     return predict(grace,ET=ET,x_predict=x_predict,start_yr=start_yr,end_yr=end_yr)


# x_predict = np.full((12*interval*n_var,len(grace.lat),len(grace.lon)),np.nan)
# start_yr = 2010
# end_yr = 2011

# interval = end_yr-start_yr+1
# n_var = 6

# pool = ThreadPool()
# pool.map(process,grace)
# pool.map(process,args)
# pool.close()
# pool.join()



In [ ]:
import scipy.io as scio
x_predict = scio.loadmat('/mnt/e/Research_life/DATA/RF_model/x_predict.mat')['name']

In [ ]:
## compute drought probability for <all of> grid
def compute_drought_probability(x_predict,n_var,interval,grace):
    n = int(len(x_predict)/n_var)
    # mon_id = 12 * 28 + 5 #2010-6
    # mon_id = 12 * 29 + 5 #2011-6
    drought_pro = np.full((12*interval,len(grace.lat),len(grace.lon)),np.nan)
    for mon_id in range(12*interval):
        for i in range(len(grace.lat)):
            for j in range(len(grace.lon)):
                X_predict = pd.DataFrame(np.array([x_predict[0:n,i,j][mon_id],x_predict[n:n*2,i,j][mon_id],
                                                    x_predict[n*2:n*3,i,j][mon_id],x_predict[n*3:n*4,i,j][mon_id],
                                                    x_predict[n*4:n*5,i,j][mon_id],x_predict[n*4:n*6,i,j][mon_id]])).T
                X_predict.columns = ['Et','TWC','Ep','pre','SM_surf','ndvi']
                try:
                    drought_pro[mon_id,i,j] = rf_model.predict_proba(X_predict)[0][1]
                except:
                    drought_pro[mon_id,i,j] = np.nan

    return drought_pro

if __name__ == '__main__':
    start_yr = 2013
    end_yr = 2015
    interval = end_yr-start_yr+1
    n_var = 6
    compute_drought_probability(x_predict,n_var,interval,grace)

In [ ]:
## multi processes
import time
from multiprocessing.dummy import Pool as ThreadPool
import threading
from multiprocessing import Process
from multiprocessing import Pool

def compute_drought_probability(x_predict,n_var,interval,grace):
    n = int(len(x_predict)/n_var)
    # mon_id = 12 * 28 + 5 #2010-6
    # mon_id = 12 * 29 + 5 #2011-6
    drought_pro = np.full((12*interval,len(grace.lat),len(grace.lon)),np.nan)
    for mon_id in range(12*interval):
        for i in range(len(grace.lat)):
            for j in range(len(grace.lon)):
                X_predict = pd.DataFrame(np.array([x_predict[0:n,i,j][mon_id],x_predict[n:n*2,i,j][mon_id],
                                                    x_predict[n*2:n*3,i,j][mon_id],x_predict[n*3:n*4,i,j][mon_id],
                                                    x_predict[n*4:n*5,i,j][mon_id],x_predict[n*4:n*6,i,j][mon_id]])).T
                X_predict.columns = ['Et','TWC','Ep','pre','SM_surf','ndvi']
                try:
                    drought_pro[mon_id,i,j] = rf_model.predict_proba(X_predict)[0][1]
                except:
                    drought_pro[mon_id,i,j] = np.nan
    return drought_pro

if __name__ == '__main__':
    start_yr = 1982
    end_yr = 2015
    interval = end_yr-start_yr+1
    n_var = 6
    pool = Pool(8)
    p = pool.apply_async(func=compute_drought_probability,args=(x_predict,n_var,interval,grace))
    pool.close()
    pool.join()
    drought_pro = p.get()

In [ ]:
## multi processes
import time
from multiprocessing.dummy import Pool as ThreadPool
import threading
from multiprocessing import Process
from multiprocessing import Pool

def compute_drought_probability(x_predict,n_var,interval):
    n = int(len(x_predict)/n_var)
    # mon_id = 12 * 28 + 5 #2010-6
    # mon_id = 12 * 29 + 5 #2011-6
    drought_pro = np.full((12*interval,48,6),np.nan)
    for mon_id in range(12*interval):
        for i in range(48):
            for j in range(6):
                X_predict = pd.DataFrame(np.array([x_predict[0:n,i,j][mon_id],x_predict[n:n*2,i,j][mon_id],
                                                    x_predict[n*2:n*3,i,j][mon_id],x_predict[n*3:n*4,i,j][mon_id],
                                                    x_predict[n*4:n*5,i,j][mon_id],x_predict[n*4:n*6,i,j][mon_id]])).T
                X_predict.columns = ['Et','TWC','Ep','pre','SM_surf','ndvi']
                try:
                    drought_pro[mon_id,i,j] = rf_model.predict_proba(X_predict)[0][1]
                except:
                    drought_pro[mon_id,i,j] = np.nan
    return drought_pro
    
if __name__ == '__main__':
    start_yr = 2015
    end_yr = 2015
    interval = end_yr-start_yr+1
    n_var = 6
    pool = Pool(4)
    for i in range(100):
        p = pool.apply_async(func=compute_drought_probability,args=(x_predict,n_var,interval))
    pool.close()
    pool.join()
    drought_pro = p.get()

In [ ]:
# ## save drought_pro dataset
# import scipy.io as io
# io.savemat('/mnt/e/Research_life/DATA/RF_model/drought_pro.mat', {'name': drought_pro})

In [ ]:
## load data
import scipy.io as scio
drought_pro = scio.loadmat('/mnt/e/Research_life/DATA/RF_model/drought_pro.mat')['name']

In [ ]:
# X_predict = X_predict.T
# X_predict.columns = ['Et','TWC','Ep','pre','SM_surf','ndvi']
rf_model.predict_proba(X_predict)[0][1]
# X_predict.T
# dro_inp.iloc[:,1:]
# dro_inp.iloc[:,0]
# X_predict

In [ ]:
np.nanmin(SPEI.values)

## Graph

In [ ]:
def Multi_spatial_graph_SPEI(lon,lat,slope1):
    from matplotlib import rcParams
    import numpy as np
    import cartopy.crs as ccrs
    import matplotlib.pyplot as plt
    import cmaps
    import cartopy.io.shapereader as shpreader
    import matplotlib as mpl
    from cartopy.io.shapereader import Reader, natural_earth
    import cartopy.feature as cfeat

    config = {
        "font.family": 'Times New Roman', # times new roman字体
        "font.size": 18, # 相当于小四大小
        "font.serif": ['SimSun'], # 宋体
        "mathtext.fontset": 'stix', # matplotlib渲染数学字体时使用的字体，和Times New Roman差别不大
        'axes.unicode_minus': False # 处理负号，即-号
    }
    rcParams.update(config)
    plt.rcParams['axes.unicode_minus']=False #用来正常显示负号
    plt.rcParams['xtick.direction'] = 'in'#将x周的刻度线方向设置向内
    plt.rcParams['ytick.direction'] = 'in'#将y轴的刻度方向设置向内
    # shp_path = "/mnt/e/Research_life/ArcGIS/USA/USA-texas/texas-state/Export_Output.shp"
    shp_path = '/mnt/e/Research_life/ArcGIS/USA/USA-texas/Texas_County_Boundaries_Detailed/County.shp'

    nx, ny = np.meshgrid(lon, lat)
    fig = plt.figure(figsize=(20,20), dpi = 300)
    proj = ccrs.PlateCarree()
    
    # 使图一colorbar映射在同一水平
    # norm1 = mpl.colors.Normalize(vmin=-abs(np.nanmax(slope1)), vmax=abs(np.nanmax(slope1)))
    ## graph one
    ax1 = fig.add_axes([0.1, 0.9, 0.5, 0.5],projection = proj)
    ax1.add_feature(cfeat.BORDERS.with_scale('50m'), linewidth=0.8, zorder=1)
    ax1.add_feature(cfeat.COASTLINE.with_scale('50m'), linewidth=0.6, zorder=1)
    # ax1.set_title('(a)',loc='left')
    gl = ax1.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.2, color='k', alpha=0.5, linestyle='--')
    gl.ylabels_right = False
    gl.xlabels_top = False
    gl.xlines = False
    gl.ylines = False
    world = shpreader.Reader(shp_path).geometries()
    ax1.add_geometries(world, ccrs.PlateCarree(),facecolor='none', edgecolor='black',zorder = 2)
    # 颜色映射，以0为分界线
    levels = np.arange(-3.5,4.5,0.5)
    c1 = ax1.contourf(nx, ny, slope1,levels=levels,cmap = cmaps.MPL_OrRd_r, transform = ccrs.PlateCarree())
    # c1.set_clim(vmin=-3.5, vmax=4)
    # c1.set_clim(vmin=-0.2, vmax=0.4)
    ax1.set_extent([-107,-92.5,25,36.5], crs = ccrs.PlateCarree())
    
    # #添加南海，实际上就是新建一个子图覆盖在之前子图的右下角
    # ax7 = fig.add_axes([0.5, 1.01, 0.12, 0.1], projection=ccrs.PlateCarree())
    # ax7.set_extent([107, 122,3,25])
    # # ax7.add_geometries(Reader(r'E:\Research_life\毕设分区\矢量图层-20210720T085947Z-001\矢量图层\国界与省界\bou2_4l.shp').geometries(),ccrs.PlateCarree(),facecolor='none',edgecolor='k',linewidth=0.8)

    # 添加色标，position定义色标位置，c1指定从c1填色图层取色，由于C3,C1的levles相同，所以色标一致，orientation设置色标为水平还是垂直，format设置色标标签格式
    position1 = fig.add_axes([0.1, 0.86, 0.5, 0.02])
    cbar=fig.colorbar(c1,cax=position1,orientation='horizontal',format='%.2f')
    cbar.set_label('$\mathrm{SPEI}$',fontsize = 24)
    return

if __name__ == '__main__':
    Lon = Lon
    Lat = Lat
    slope = np.zeros((62,42))
    Multi_spatial_graph_SPEI(Lon,Lat,slope)

In [ ]:
def Multi_spatial_graph(lon,lat,slope1):
    from matplotlib import rcParams
    import numpy as np
    import cartopy.crs as ccrs
    import matplotlib.pyplot as plt
    import cmaps
    import cartopy.io.shapereader as shpreader
    import matplotlib as mpl
    from cartopy.io.shapereader import Reader, natural_earth
    import cartopy.feature as cfeat
    from matplotlib.colors import ListedColormap 

    config = {
        "font.family": 'Times New Roman', # times new roman字体
        "font.size": 18, # 相当于小四大小
        "font.serif": ['SimSun'], # 宋体
        "mathtext.fontset": 'stix', # matplotlib渲染数学字体时使用的字体，和Times New Roman差别不大
        'axes.unicode_minus': False # 处理负号，即-号
    }
    rcParams.update(config)
    plt.rcParams['axes.unicode_minus']=False #用来正常显示负号
    plt.rcParams['xtick.direction'] = 'in'#将x周的刻度线方向设置向内
    plt.rcParams['ytick.direction'] = 'in'#将y轴的刻度方向设置向内
    # shp_path = "/mnt/e/Research_life/ArcGIS/USA/USA-texas/texas-state/Export_Output.shp"
    shp_path = '/mnt/e/Research_life/ArcGIS/USA/USA-texas/Texas_County_Boundaries_Detailed/County.shp'
    nx, ny = np.meshgrid(lon, lat)
    fig = plt.figure(figsize=(20,20), dpi = 300)
    proj = ccrs.PlateCarree()
    
    # 使图一colorbar映射在同一水平
    # norm1 = mpl.colors.Normalize(vmin=0.2, vmax=1)
    # norm1 = mpl.colors.BoundaryNorm(np.arange(0.2,1.1,0.1),cmaps.MPL_OrRd.N)
    # newcolors = cmaps.MPL_OrRd(np.linspace(0,1,10))
    # newcmap=ListedColormap(newcolors[::1]) 


    ## graph one
    ax1 = fig.add_axes([0.1, 0.9, 0.5, 0.5],projection = proj)
    ax1.add_feature(cfeat.BORDERS.with_scale('50m'), linewidth=0.8, zorder=1)
    ax1.add_feature(cfeat.COASTLINE.with_scale('50m'), linewidth=0.6, zorder=1)
    # ax1.set_title('(a)',loc='left')
    gl = ax1.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.2, color='k', alpha=0.5, linestyle='--')
    gl.ylabels_right = False
    gl.xlabels_top = False
    gl.xlines = False
    gl.ylines = False
    world = shpreader.Reader(shp_path).geometries()
    ax1.add_geometries(world, ccrs.PlateCarree(),facecolor='none', edgecolor='black',zorder = 2)
    # 颜色映射，以0为分界线
    levels = np.arange(0,1.1,0.1)
    c1 = ax1.contourf(nx, ny, slope1,levels=levels,cmap = cmaps.MPL_OrRd, transform = ccrs.PlateCarree())
    # c1 = ax1.contourf(nx, ny, slope1,levels=levels,cmap = newcmap, transform = ccrs.PlateCarree())

    c1.set_clim(vmin=0.3, vmax=1)
    # c1.set_clim(vmin=-0.2, vmax=0.4)
    ax1.set_extent([-107,-92.5,25,36.5], crs = ccrs.PlateCarree())
    
    # 添加色标，position定义色标位置，c1指定从c1填色图层取色，由于C3,C1的levles相同，所以色标一致，orientation设置色标为水平还是垂直，format设置色标标签格式
    position1 = fig.add_axes([0.1, 0.86, 0.5, 0.02])
    cbar=fig.colorbar(c1,cax=position1,orientation='horizontal',format='%.2f')
    cbar.set_label('$\mathrm{dorught_pro}$',fontsize = 24)
    return

if __name__ == '__main__':
    Lon = Lon
    Lat = Lat
    # slope = np.zeros((62,42))
    Multi_spatial_graph(Lon,Lat,drought_pro[0,::])
    # Multi_spatial_graph(Lon,Lat,slope)

In [ ]:
def Multi_spatial_graph_PDSI(lon,lat,slope1,start_yr,end_yr):
    from matplotlib import rcParams
    import numpy as np
    import cartopy.crs as ccrs
    import matplotlib.pyplot as plt
    import cmaps
    import cartopy.io.shapereader as shpreader
    import matplotlib as mpl
    from cartopy.io.shapereader import Reader, natural_earth
    import cartopy.feature as cfeat

    config = {
        "font.family": 'Times New Roman', # times new roman字体
        "font.size": 30, # 相当于小四大小
        "font.serif": ['SimSun'], # 宋体
        "mathtext.fontset": 'stix', # matplotlib渲染数学字体时使用的字体，和Times New Roman差别不大
        'axes.unicode_minus': False # 处理负号，即-号
    }
    rcParams.update(config)
    plt.rcParams['axes.unicode_minus']=False #用来正常显示负号
    plt.rcParams['xtick.direction'] = 'in'#将x周的刻度线方向设置向内
    plt.rcParams['ytick.direction'] = 'in'#将y轴的刻度方向设置向内
    shp_path = '/mnt/e/Research_life/ArcGIS/USA/USA-texas/Texas_County_Boundaries_Detailed/County.shp'

    pdsi = slope1.loc[str(start_yr):str(end_yr),::].values
    world = shpreader.Reader(shp_path).geometries()
    levels = np.arange(-8,9,1)

    nx, ny = np.meshgrid(lon, lat)
    fig = plt.figure(figsize=(20,20), dpi = 300)
    proj = ccrs.PlateCarree()
    
    # 使图一colorbar映射在同一水平
    # norm1 = mpl.colors.Normalize(vmin=-abs(np.nanmax(slope1)), vmax=abs(np.nanmax(slope1)))
    ## graph one
    ax1 = fig.add_axes([0.1,3, 0.5, 0.5],projection = proj)
    ax1.add_feature(cfeat.BORDERS.with_scale('50m'), linewidth=0.8, zorder=1)
    ax1.add_feature(cfeat.COASTLINE.with_scale('50m'), linewidth=0.6, zorder=1)
    # ax1.set_title('(a)',loc='left')
    gl = ax1.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.2, color='k', alpha=0.5, linestyle='--')
    gl.ylabels_right = False
    gl.xlabels_top = False
    gl.xlines = False
    gl.ylines = False
    ax1.add_geometries(world, ccrs.PlateCarree(),facecolor='none', edgecolor='black',zorder = 2)
    # 颜色映射，以0为分界线
    c1 = ax1.contourf(nx, ny, pdsi[0,::],levels=levels,cmap = cmaps.MPL_OrRd_r, transform = ccrs.PlateCarree())
    # c1.set_clim(vmin=-3.5, vmax=4)
    # c1.set_clim(vmin=-0.2, vmax=0.4)
    ax1.set_extent([-107,-92.5,25,36.5], crs = ccrs.PlateCarree())
    
    ax2 = fig.add_axes([0.7, 3, 0.5, 0.5],projection = proj)
    ax2.add_feature(cfeat.BORDERS.with_scale('50m'), linewidth=0.8, zorder=1)
    ax2.add_feature(cfeat.COASTLINE.with_scale('50m'), linewidth=0.6, zorder=1)
    gl = ax2.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.2, color='k', alpha=0.5, linestyle='--')
    gl.ylabels_right = False
    gl.xlabels_top = False
    gl.xlines = False
    gl.ylines = False
    world = shpreader.Reader(shp_path).geometries()
    ax2.add_geometries(world, ccrs.PlateCarree(),facecolor='none', edgecolor='black',zorder = 2)
    c2 = ax2.contourf(nx, ny, pdsi[1,::],levels=levels,cmap = cmaps.MPL_OrRd_r, transform = ccrs.PlateCarree(),zorder=1)
    ax2.set_extent([-107,-92.5,25,36.5], crs = ccrs.PlateCarree())

    ax3 = fig.add_axes([1.3, 3, 0.5, 0.5],projection = proj)
    ax3.add_feature(cfeat.BORDERS.with_scale('50m'), linewidth=0.8, zorder=1)
    ax3.add_feature(cfeat.COASTLINE.with_scale('50m'), linewidth=0.6, zorder=1)
    gl = ax3.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.2, color='k', alpha=0.5, linestyle='--')
    gl.ylabels_right = False
    gl.xlabels_top = False
    gl.xlines = False
    gl.ylines = False
    world = shpreader.Reader(shp_path).geometries()
    ax3.add_geometries(world, ccrs.PlateCarree(),facecolor='none', edgecolor='black',zorder = 2)
    c3 = ax3.contourf(nx, ny, pdsi[2,::],levels=levels,cmap = cmaps.MPL_OrRd_r, transform = ccrs.PlateCarree())
    ax3.set_extent([-107,-92.5,25,36.5], crs = ccrs.PlateCarree())

    ax4 = fig.add_axes([1.9, 3, 0.5, 0.5],projection = proj)
    ax4.add_feature(cfeat.BORDERS.with_scale('50m'), linewidth=0.8, zorder=1)
    ax4.add_feature(cfeat.COASTLINE.with_scale('50m'), linewidth=0.6, zorder=1)
    gl = ax4.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.2, color='k', alpha=0.5, linestyle='--')
    gl.ylabels_right = False
    gl.xlabels_top = False
    gl.xlines = False
    gl.ylines = False
    world = shpreader.Reader(shp_path).geometries()
    ax4.add_geometries(world, ccrs.PlateCarree(),facecolor='none', edgecolor='black',zorder = 2)
    c4 = ax4.contourf(nx, ny, pdsi[3,::],levels=levels,cmap = cmaps.MPL_OrRd_r, transform = ccrs.PlateCarree())
    ax4.set_extent([-107,-92.5,25,36.5], crs = ccrs.PlateCarree())

    ax5 = fig.add_axes([2.5, 3, 0.5, 0.5],projection = proj)
    ax5.add_feature(cfeat.BORDERS.with_scale('50m'), linewidth=0.8, zorder=1)
    ax5.add_feature(cfeat.COASTLINE.with_scale('50m'), linewidth=0.6, zorder=1)
    gl = ax5.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.2, color='k', alpha=0.5, linestyle='--')
    gl.ylabels_right = False
    gl.xlabels_top = False
    gl.xlines = False
    gl.ylines = False
    world = shpreader.Reader(shp_path).geometries()
    ax5.add_geometries(world, ccrs.PlateCarree(),facecolor='none', edgecolor='black',zorder = 2)
    c5 = ax5.contourf(nx, ny, pdsi[4,::],levels=levels,cmap = cmaps.MPL_OrRd_r, transform = ccrs.PlateCarree())
    ax5.set_extent([-107,-92.5,25,36.5], crs = ccrs.PlateCarree())

    ax11 = fig.add_axes([0.1, 2.5, 0.5, 0.5],projection = proj)
    ax11.add_feature(cfeat.BORDERS.with_scale('50m'), linewidth=0.8, zorder=1)
    ax11.add_feature(cfeat.COASTLINE.with_scale('50m'), linewidth=0.6, zorder=1)
    gl = ax11.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.2, color='k', alpha=0.5, linestyle='--')
    gl.ylabels_right = False
    gl.xlabels_top = False
    gl.xlines = False
    gl.ylines = False
    world = shpreader.Reader(shp_path).geometries()
    ax11.add_geometries(world, ccrs.PlateCarree(),facecolor='none', edgecolor='black',zorder = 2)
    c11 = ax11.contourf(nx, ny, pdsi[5,::],levels=levels,cmap = cmaps.MPL_OrRd_r, transform = ccrs.PlateCarree())
    ax11.set_extent([-107,-92.5,25,36.5], crs = ccrs.PlateCarree())

    ax21 = fig.add_axes([0.7, 2.5, 0.5, 0.5],projection = proj)
    ax21.add_feature(cfeat.BORDERS.with_scale('50m'), linewidth=0.8, zorder=1)
    ax21.add_feature(cfeat.COASTLINE.with_scale('50m'), linewidth=0.6, zorder=1)
    gl = ax21.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.2, color='k', alpha=0.5, linestyle='--')
    gl.ylabels_right = False
    gl.xlabels_top = False
    gl.xlines = False
    gl.ylines = False
    world = shpreader.Reader(shp_path).geometries()
    ax21.add_geometries(world, ccrs.PlateCarree(),facecolor='none', edgecolor='black',zorder = 2)
    c21 = ax21.contourf(nx, ny, pdsi[6,::],levels=levels,cmap = cmaps.MPL_OrRd_r, transform = ccrs.PlateCarree(),zorder=1)
    ax21.set_extent([-107,-92.5,25,36.5], crs = ccrs.PlateCarree())

    ax31 = fig.add_axes([1.3, 2.5, 0.5, 0.5],projection = proj)
    ax31.add_feature(cfeat.BORDERS.with_scale('50m'), linewidth=0.8, zorder=1)
    ax31.add_feature(cfeat.COASTLINE.with_scale('50m'), linewidth=0.6, zorder=1)
    gl = ax31.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.2, color='k', alpha=0.5, linestyle='--')
    gl.ylabels_right = False
    gl.xlabels_top = False
    gl.xlines = False
    gl.ylines = False
    world = shpreader.Reader(shp_path).geometries()
    ax31.add_geometries(world, ccrs.PlateCarree(),facecolor='none', edgecolor='black',zorder = 2)
    c31 = ax31.contourf(nx, ny, pdsi[7,::],levels=levels,cmap = cmaps.MPL_OrRd_r, transform = ccrs.PlateCarree())
    ax31.set_extent([-107,-92.5,25,36.5], crs = ccrs.PlateCarree())

    ax41 = fig.add_axes([1.9, 2.5, 0.5, 0.5],projection = proj)
    ax41.add_feature(cfeat.BORDERS.with_scale('50m'), linewidth=0.8, zorder=1)
    ax41.add_feature(cfeat.COASTLINE.with_scale('50m'), linewidth=0.6, zorder=1)
    gl = ax41.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.2, color='k', alpha=0.5, linestyle='--')
    gl.ylabels_right = False
    gl.xlabels_top = False
    gl.xlines = False
    gl.ylines = False
    world = shpreader.Reader(shp_path).geometries()
    ax41.add_geometries(world, ccrs.PlateCarree(),facecolor='none', edgecolor='black',zorder = 2)
    c41 = ax41.contourf(nx, ny, pdsi[8,::],levels=levels,cmap = cmaps.MPL_OrRd_r, transform = ccrs.PlateCarree())
    ax41.set_extent([-107,-92.5,25,36.5], crs = ccrs.PlateCarree())

    ax51 = fig.add_axes([2.5, 2.5, 0.5, 0.5],projection = proj)
    ax51.add_feature(cfeat.BORDERS.with_scale('50m'), linewidth=0.8, zorder=1)
    ax51.add_feature(cfeat.COASTLINE.with_scale('50m'), linewidth=0.6, zorder=1)
    gl = ax51.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.2, color='k', alpha=0.5, linestyle='--')
    gl.ylabels_right = False
    gl.xlabels_top = False
    gl.xlines = False
    gl.ylines = False
    world = shpreader.Reader(shp_path).geometries()
    ax51.add_geometries(world, ccrs.PlateCarree(),facecolor='none', edgecolor='black',zorder = 2)
    c51 = ax51.contourf(nx, ny, pdsi[9,::],levels=levels,cmap = cmaps.MPL_OrRd_r, transform = ccrs.PlateCarree())
    ax51.set_extent([-107,-92.5,25,36.5], crs = ccrs.PlateCarree())

    ax12 = fig.add_axes([0.1, 2, 0.5, 0.5],projection = proj)
    ax12.add_feature(cfeat.BORDERS.with_scale('50m'), linewidth=0.8, zorder=1)
    ax12.add_feature(cfeat.COASTLINE.with_scale('50m'), linewidth=0.6, zorder=1)
    gl = ax12.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.2, color='k', alpha=0.5, linestyle='--')
    gl.ylabels_right = False
    gl.xlabels_top = False
    gl.xlines = False
    gl.ylines = False
    world = shpreader.Reader(shp_path).geometries()
    ax12.add_geometries(world, ccrs.PlateCarree(),facecolor='none', edgecolor='black',zorder = 2)
    c12 = ax12.contourf(nx, ny, pdsi[10,::],levels=levels,cmap = cmaps.MPL_OrRd_r, transform = ccrs.PlateCarree())
    ax12.set_extent([-107,-92.5,25,36.5], crs = ccrs.PlateCarree())

    ax22 = fig.add_axes([0.7, 2, 0.5, 0.5],projection = proj)
    ax22.add_feature(cfeat.BORDERS.with_scale('50m'), linewidth=0.8, zorder=1)
    ax22.add_feature(cfeat.COASTLINE.with_scale('50m'), linewidth=0.6, zorder=1)
    gl = ax22.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.2, color='k', alpha=0.5, linestyle='--')
    gl.ylabels_right = False
    gl.xlabels_top = False
    gl.xlines = False
    gl.ylines = False
    world = shpreader.Reader(shp_path).geometries()
    ax22.add_geometries(world, ccrs.PlateCarree(),facecolor='none', edgecolor='black',zorder = 2)
    c22 = ax22.contourf(nx, ny, pdsi[11,::],levels=levels,cmap = cmaps.MPL_OrRd_r, transform = ccrs.PlateCarree(),zorder=1)
    ax22.set_extent([-107,-92.5,25,36.5], crs = ccrs.PlateCarree())

    ax32 = fig.add_axes([1.3, 2, 0.5, 0.5],projection = proj)
    ax32.add_feature(cfeat.BORDERS.with_scale('50m'), linewidth=0.8, zorder=1)
    ax32.add_feature(cfeat.COASTLINE.with_scale('50m'), linewidth=0.6, zorder=1)
    gl = ax32.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.2, color='k', alpha=0.5, linestyle='--')
    gl.ylabels_right = False
    gl.xlabels_top = False
    gl.xlines = False
    gl.ylines = False
    world = shpreader.Reader(shp_path).geometries()
    ax32.add_geometries(world, ccrs.PlateCarree(),facecolor='none', edgecolor='black',zorder = 2)
    c32 = ax32.contourf(nx, ny, pdsi[12,::],levels=levels,cmap = cmaps.MPL_OrRd_r, transform = ccrs.PlateCarree())
    ax32.set_extent([-107,-92.5,25,36.5], crs = ccrs.PlateCarree())

    ax42 = fig.add_axes([1.9, 2, 0.5, 0.5],projection = proj)
    ax42.add_feature(cfeat.BORDERS.with_scale('50m'), linewidth=0.8, zorder=1)
    ax42.add_feature(cfeat.COASTLINE.with_scale('50m'), linewidth=0.6, zorder=1)
    gl = ax42.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.2, color='k', alpha=0.5, linestyle='--')
    gl.ylabels_right = False
    gl.xlabels_top = False
    gl.xlines = False
    gl.ylines = False
    world = shpreader.Reader(shp_path).geometries()
    ax42.add_geometries(world, ccrs.PlateCarree(),facecolor='none', edgecolor='black',zorder = 2)
    c42 = ax42.contourf(nx, ny, pdsi[13,::],levels=levels,cmap = cmaps.MPL_OrRd_r, transform = ccrs.PlateCarree())
    ax42.set_extent([-107,-92.5,25,36.5], crs = ccrs.PlateCarree())

    ax52 = fig.add_axes([2.5, 2, 0.5, 0.5],projection = proj)
    ax52.add_feature(cfeat.BORDERS.with_scale('50m'), linewidth=0.8, zorder=1)
    ax52.add_feature(cfeat.COASTLINE.with_scale('50m'), linewidth=0.6, zorder=1)
    gl = ax52.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.2, color='k', alpha=0.5, linestyle='--')
    gl.ylabels_right = False
    gl.xlabels_top = False
    gl.xlines = False
    gl.ylines = False
    world = shpreader.Reader(shp_path).geometries()
    ax52.add_geometries(world, ccrs.PlateCarree(),facecolor='none', edgecolor='black',zorder = 2)
    c52 = ax52.contourf(nx, ny, pdsi[14,::],levels=levels,cmap = cmaps.MPL_OrRd_r, transform = ccrs.PlateCarree())
    ax52.set_extent([-107,-92.5,25,36.5], crs = ccrs.PlateCarree())

    ax13 = fig.add_axes([0.1, 1.5, 0.5, 0.5],projection = proj)
    ax13.add_feature(cfeat.BORDERS.with_scale('50m'), linewidth=0.8, zorder=1)
    ax13.add_feature(cfeat.COASTLINE.with_scale('50m'), linewidth=0.6, zorder=1)
    gl = ax13.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.2, color='k', alpha=0.5, linestyle='--')
    gl.ylabels_right = False
    gl.xlabels_top = False
    gl.xlines = False
    gl.ylines = False
    world = shpreader.Reader(shp_path).geometries()
    ax13.add_geometries(world, ccrs.PlateCarree(),facecolor='none', edgecolor='black',zorder = 2)
    c13 = ax13.contourf(nx, ny, pdsi[15,::],levels=levels,cmap = cmaps.MPL_OrRd_r, transform = ccrs.PlateCarree())
    ax13.set_extent([-107,-92.5,25,36.5], crs = ccrs.PlateCarree())

    ax23 = fig.add_axes([0.7, 1.5, 0.5, 0.5],projection = proj)
    ax23.add_feature(cfeat.BORDERS.with_scale('50m'), linewidth=0.8, zorder=1)
    ax23.add_feature(cfeat.COASTLINE.with_scale('50m'), linewidth=0.6, zorder=1)
    gl = ax23.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.2, color='k', alpha=0.5, linestyle='--')
    gl.ylabels_right = False
    gl.xlabels_top = False
    gl.xlines = False
    gl.ylines = False
    world = shpreader.Reader(shp_path).geometries()
    ax23.add_geometries(world, ccrs.PlateCarree(),facecolor='none', edgecolor='black',zorder = 2)
    c23 = ax23.contourf(nx, ny, pdsi[16,::],levels=levels,cmap = cmaps.MPL_OrRd_r, transform = ccrs.PlateCarree(),zorder=1)
    ax23.set_extent([-107,-92.5,25,36.5], crs = ccrs.PlateCarree())

    ax33 = fig.add_axes([1.3, 1.5, 0.5, 0.5],projection = proj)
    ax33.add_feature(cfeat.BORDERS.with_scale('50m'), linewidth=0.8, zorder=1)
    ax33.add_feature(cfeat.COASTLINE.with_scale('50m'), linewidth=0.6, zorder=1)
    gl = ax33.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.2, color='k', alpha=0.5, linestyle='--')
    gl.ylabels_right = False
    gl.xlabels_top = False
    gl.xlines = False
    gl.ylines = False
    world = shpreader.Reader(shp_path).geometries()
    ax33.add_geometries(world, ccrs.PlateCarree(),facecolor='none', edgecolor='black',zorder = 2)
    c33 = ax33.contourf(nx, ny, pdsi[17,::],levels=levels,cmap = cmaps.MPL_OrRd_r, transform = ccrs.PlateCarree())
    ax33.set_extent([-107,-92.5,25,36.5], crs = ccrs.PlateCarree())

    ax43 = fig.add_axes([1.9, 1.5, 0.5, 0.5],projection = proj)
    ax43.add_feature(cfeat.BORDERS.with_scale('50m'), linewidth=0.8, zorder=1)
    ax43.add_feature(cfeat.COASTLINE.with_scale('50m'), linewidth=0.6, zorder=1)
    gl = ax43.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.2, color='k', alpha=0.5, linestyle='--')
    gl.ylabels_right = False
    gl.xlabels_top = False
    gl.xlines = False
    gl.ylines = False
    world = shpreader.Reader(shp_path).geometries()
    ax43.add_geometries(world, ccrs.PlateCarree(),facecolor='none', edgecolor='black',zorder = 2)
    c43 = ax43.contourf(nx, ny, pdsi[18,::],levels=levels,cmap = cmaps.MPL_OrRd_r, transform = ccrs.PlateCarree())
    ax43.set_extent([-107,-92.5,25,36.5], crs = ccrs.PlateCarree())

    ax53 = fig.add_axes([2.5, 1.5, 0.5, 0.5],projection = proj)
    ax53.add_feature(cfeat.BORDERS.with_scale('50m'), linewidth=0.8, zorder=1)
    ax53.add_feature(cfeat.COASTLINE.with_scale('50m'), linewidth=0.6, zorder=1)
    gl = ax53.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.2, color='k', alpha=0.5, linestyle='--')
    gl.ylabels_right = False
    gl.xlabels_top = False
    gl.xlines = False
    gl.ylines = False
    world = shpreader.Reader(shp_path).geometries()
    ax53.add_geometries(world, ccrs.PlateCarree(),facecolor='none', edgecolor='black',zorder = 2)
    c53 = ax53.contourf(nx, ny, pdsi[19,::],levels=levels,cmap = cmaps.MPL_OrRd_r, transform = ccrs.PlateCarree())
    ax53.set_extent([-107,-92.5,25,36.5], crs = ccrs.PlateCarree())

    ax14 = fig.add_axes([0.1, 1, 0.5, 0.5],projection = proj)
    ax14.add_feature(cfeat.BORDERS.with_scale('50m'), linewidth=0.8, zorder=1)
    ax14.add_feature(cfeat.COASTLINE.with_scale('50m'), linewidth=0.6, zorder=1)
    gl = ax14.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.2, color='k', alpha=0.5, linestyle='--')
    gl.ylabels_right = False
    gl.xlabels_top = False
    gl.xlines = False
    gl.ylines = False
    world = shpreader.Reader(shp_path).geometries()
    ax14.add_geometries(world, ccrs.PlateCarree(),facecolor='none', edgecolor='black',zorder = 2)
    c14 = ax14.contourf(nx, ny, pdsi[20,::],levels=levels,cmap = cmaps.MPL_OrRd_r, transform = ccrs.PlateCarree())
    ax14.set_extent([-107,-92.5,25,36.5], crs = ccrs.PlateCarree())

    ax24 = fig.add_axes([0.7, 1, 0.5, 0.5],projection = proj)
    ax24.add_feature(cfeat.BORDERS.with_scale('50m'), linewidth=0.8, zorder=1)
    ax24.add_feature(cfeat.COASTLINE.with_scale('50m'), linewidth=0.6, zorder=1)
    gl = ax24.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.2, color='k', alpha=0.5, linestyle='--')
    gl.ylabels_right = False
    gl.xlabels_top = False
    gl.xlines = False
    gl.ylines = False
    world = shpreader.Reader(shp_path).geometries()
    ax24.add_geometries(world, ccrs.PlateCarree(),facecolor='none', edgecolor='black',zorder = 2)
    c24 = ax24.contourf(nx, ny, pdsi[21,::],levels=levels,cmap = cmaps.MPL_OrRd_r, transform = ccrs.PlateCarree(),zorder=1)
    ax24.set_extent([-107,-92.5,25,36.5], crs = ccrs.PlateCarree())

    ax34 = fig.add_axes([1.3, 1, 0.5, 0.5],projection = proj)
    ax34.add_feature(cfeat.BORDERS.with_scale('50m'), linewidth=0.8, zorder=1)
    ax34.add_feature(cfeat.COASTLINE.with_scale('50m'), linewidth=0.6, zorder=1)
    gl = ax34.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.2, color='k', alpha=0.5, linestyle='--')
    gl.ylabels_right = False
    gl.xlabels_top = False
    gl.xlines = False
    gl.ylines = False
    world = shpreader.Reader(shp_path).geometries()
    ax34.add_geometries(world, ccrs.PlateCarree(),facecolor='none', edgecolor='black',zorder = 2)
    c34 = ax34.contourf(nx, ny, pdsi[22,::],levels=levels,cmap = cmaps.MPL_OrRd_r, transform = ccrs.PlateCarree())
    ax34.set_extent([-107,-92.5,25,36.5], crs = ccrs.PlateCarree())

    ax44 = fig.add_axes([1.9, 1, 0.5, 0.5],projection = proj)
    ax44.add_feature(cfeat.BORDERS.with_scale('50m'), linewidth=0.8, zorder=1)
    ax44.add_feature(cfeat.COASTLINE.with_scale('50m'), linewidth=0.6, zorder=1)
    gl = ax44.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.2, color='k', alpha=0.5, linestyle='--')
    gl.ylabels_right = False
    gl.xlabels_top = False
    gl.xlines = False
    gl.ylines = False
    world = shpreader.Reader(shp_path).geometries()
    ax44.add_geometries(world, ccrs.PlateCarree(),facecolor='none', edgecolor='black',zorder = 2)
    c44 = ax44.contourf(nx, ny, pdsi[23,::],levels=levels,cmap = cmaps.MPL_OrRd_r, transform = ccrs.PlateCarree())
    ax44.set_extent([-107,-92.5,25,36.5], crs = ccrs.PlateCarree())

    # ax54 = fig.add_axes([2.5, 1, 0.5, 0.5],projection = proj)
    # ax54.add_feature(cfeat.BORDERS.with_scale('50m'), linewidth=0.8, zorder=1)
    # ax54.add_feature(cfeat.COASTLINE.with_scale('50m'), linewidth=0.6, zorder=1)
    # gl = ax54.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.2, color='k', alpha=0.5, linestyle='--')
    # gl.ylabels_right = False
    # gl.xlabels_top = False
    # gl.xlines = False
    # gl.ylines = False
    # world = shpreader.Reader(shp_path).geometries()
    # ax54.add_geometries(world, ccrs.PlateCarree(),facecolor='none', edgecolor='black',zorder = 2)
    # c54 = ax54.contourf(nx, ny, pdsi[24,::],levels=levels,cmap = cmaps.MPL_OrRd_r, transform = ccrs.PlateCarree())
    # ax54.set_extent([-107,-92.5,25,36.5], crs = ccrs.PlateCarree())



    # #添加南海，实际上就是新建一个子图覆盖在之前子图的右下角
    # ax7 = fig.add_axes([0.5, 1.01, 0.12, 0.1], projection=ccrs.PlateCarree())
    # ax7.set_extent([107, 122,3,25])
    # # ax7.add_geometries(Reader(r'E:\Research_life\毕设分区\矢量图层-20210720T085947Z-001\矢量图层\国界与省界\bou2_4l.shp').geometries(),ccrs.PlateCarree(),facecolor='none',edgecolor='k',linewidth=0.8)

    # 添加色标，position定义色标位置，c1指定从c1填色图层取色，由于C3,C1的levles相同，所以色标一致，orientation设置色标为水平还是垂直，format设置色标标签格式
    position1 = fig.add_axes([0.2, 0.9, 2.4, 0.05])
    cbar=fig.colorbar(c1,cax=position1,orientation='horizontal',format='%.2f')
    cbar.set_label('$\mathrm{PDSI}$',fontsize = 54)
    return

# if __name__ == '__main__':
#     Lon = Lon
#     Lat = Lat
#     # slope = np.zeros((62,42))
#     # Multi_spatial_graph_PDSI(pdsi.lon.values,pdsi.lat.values,pdsi,2010,2011)
#     Multi_spatial_graph_PDSI(drought_pro_nc.lon.values,drought_pro_nc,drought_pro_nc,2010,2011)



In [ ]:
def Multi_spatial_graph_drought_pro(lon,lat,slope1,start_yr,end_yr):
    from matplotlib import rcParams
    import numpy as np
    import cartopy.crs as ccrs
    import matplotlib.pyplot as plt
    import cmaps
    import cartopy.io.shapereader as shpreader
    import matplotlib as mpl
    from cartopy.io.shapereader import Reader, natural_earth
    import cartopy.feature as cfeat

    config = {
        "font.family": 'Times New Roman', # times new roman字体
        "font.size": 30, # 相当于小四大小
        "font.serif": ['SimSun'], # 宋体
        "mathtext.fontset": 'stix', # matplotlib渲染数学字体时使用的字体，和Times New Roman差别不大
        'axes.unicode_minus': False # 处理负号，即-号
    }
    rcParams.update(config)
    plt.rcParams['axes.unicode_minus']=False #用来正常显示负号
    plt.rcParams['xtick.direction'] = 'in'#将x周的刻度线方向设置向内
    plt.rcParams['ytick.direction'] = 'in'#将y轴的刻度方向设置向内
    shp_path = '/mnt/e/Research_life/ArcGIS/USA/USA-texas/Texas_County_Boundaries_Detailed/County.shp'

    pdsi = slope1.loc[str(start_yr):str(end_yr),::].values
    world = shpreader.Reader(shp_path).geometries()
    levels = np.arange(0,1.1,0.1)
    cmap = cmaps.MPL_OrRd
    nx, ny = np.meshgrid(lon, lat)
    fig = plt.figure(figsize=(20,20), dpi = 300)
    proj = ccrs.PlateCarree()
    
    # 使图一colorbar映射在同一水平
    # norm1 = mpl.colors.Normalize(vmin=-abs(np.nanmax(slope1)), vmax=abs(np.nanmax(slope1)))
    ## graph one
    ax1 = fig.add_axes([0.1,3, 0.5, 0.5],projection = proj)
    ax1.add_feature(cfeat.BORDERS.with_scale('50m'), linewidth=0.8, zorder=1)
    ax1.add_feature(cfeat.COASTLINE.with_scale('50m'), linewidth=0.6, zorder=1)
    # ax1.set_title('(a)',loc='left')
    gl = ax1.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.2, color='k', alpha=0.5, linestyle='--')
    gl.ylabels_right = False
    gl.xlabels_top = False
    gl.xlines = False
    gl.ylines = False
    ax1.add_geometries(world, ccrs.PlateCarree(),facecolor='none', edgecolor='black',zorder = 2)
    # 颜色映射，以0为分界线
    c1 = ax1.contourf(nx, ny, pdsi[0,::],levels=levels,cmap = cmap, transform = ccrs.PlateCarree())
    # c1.set_clim(vmin=-3.5, vmax=4)
    # c1.set_clim(vmin=-0.2, vmax=0.4)
    ax1.set_extent([-107,-92.5,25,36.5], crs = ccrs.PlateCarree())
    
    ax2 = fig.add_axes([0.7, 3, 0.5, 0.5],projection = proj)
    ax2.add_feature(cfeat.BORDERS.with_scale('50m'), linewidth=0.8, zorder=1)
    ax2.add_feature(cfeat.COASTLINE.with_scale('50m'), linewidth=0.6, zorder=1)
    gl = ax2.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.2, color='k', alpha=0.5, linestyle='--')
    gl.ylabels_right = False
    gl.xlabels_top = False
    gl.xlines = False
    gl.ylines = False
    world = shpreader.Reader(shp_path).geometries()
    ax2.add_geometries(world, ccrs.PlateCarree(),facecolor='none', edgecolor='black',zorder = 2)
    c2 = ax2.contourf(nx, ny, pdsi[1,::],levels=levels,cmap = cmap, transform = ccrs.PlateCarree(),zorder=1)
    ax2.set_extent([-107,-92.5,25,36.5], crs = ccrs.PlateCarree())

    ax3 = fig.add_axes([1.3, 3, 0.5, 0.5],projection = proj)
    ax3.add_feature(cfeat.BORDERS.with_scale('50m'), linewidth=0.8, zorder=1)
    ax3.add_feature(cfeat.COASTLINE.with_scale('50m'), linewidth=0.6, zorder=1)
    gl = ax3.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.2, color='k', alpha=0.5, linestyle='--')
    gl.ylabels_right = False
    gl.xlabels_top = False
    gl.xlines = False
    gl.ylines = False
    world = shpreader.Reader(shp_path).geometries()
    ax3.add_geometries(world, ccrs.PlateCarree(),facecolor='none', edgecolor='black',zorder = 2)
    c3 = ax3.contourf(nx, ny, pdsi[2,::],levels=levels,cmap = cmap, transform = ccrs.PlateCarree())
    ax3.set_extent([-107,-92.5,25,36.5], crs = ccrs.PlateCarree())

    ax4 = fig.add_axes([1.9, 3, 0.5, 0.5],projection = proj)
    ax4.add_feature(cfeat.BORDERS.with_scale('50m'), linewidth=0.8, zorder=1)
    ax4.add_feature(cfeat.COASTLINE.with_scale('50m'), linewidth=0.6, zorder=1)
    gl = ax4.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.2, color='k', alpha=0.5, linestyle='--')
    gl.ylabels_right = False
    gl.xlabels_top = False
    gl.xlines = False
    gl.ylines = False
    world = shpreader.Reader(shp_path).geometries()
    ax4.add_geometries(world, ccrs.PlateCarree(),facecolor='none', edgecolor='black',zorder = 2)
    c4 = ax4.contourf(nx, ny, pdsi[3,::],levels=levels,cmap = cmap, transform = ccrs.PlateCarree())
    ax4.set_extent([-107,-92.5,25,36.5], crs = ccrs.PlateCarree())

    ax5 = fig.add_axes([2.5, 3, 0.5, 0.5],projection = proj)
    ax5.add_feature(cfeat.BORDERS.with_scale('50m'), linewidth=0.8, zorder=1)
    ax5.add_feature(cfeat.COASTLINE.with_scale('50m'), linewidth=0.6, zorder=1)
    gl = ax5.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.2, color='k', alpha=0.5, linestyle='--')
    gl.ylabels_right = False
    gl.xlabels_top = False
    gl.xlines = False
    gl.ylines = False
    world = shpreader.Reader(shp_path).geometries()
    ax5.add_geometries(world, ccrs.PlateCarree(),facecolor='none', edgecolor='black',zorder = 2)
    c5 = ax5.contourf(nx, ny, pdsi[4,::],levels=levels,cmap = cmap, transform = ccrs.PlateCarree())
    ax5.set_extent([-107,-92.5,25,36.5], crs = ccrs.PlateCarree())

    ax11 = fig.add_axes([0.1, 2.5, 0.5, 0.5],projection = proj)
    ax11.add_feature(cfeat.BORDERS.with_scale('50m'), linewidth=0.8, zorder=1)
    ax11.add_feature(cfeat.COASTLINE.with_scale('50m'), linewidth=0.6, zorder=1)
    gl = ax11.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.2, color='k', alpha=0.5, linestyle='--')
    gl.ylabels_right = False
    gl.xlabels_top = False
    gl.xlines = False
    gl.ylines = False
    world = shpreader.Reader(shp_path).geometries()
    ax11.add_geometries(world, ccrs.PlateCarree(),facecolor='none', edgecolor='black',zorder = 2)
    c11 = ax11.contourf(nx, ny, pdsi[5,::],levels=levels,cmap = cmap, transform = ccrs.PlateCarree())
    ax11.set_extent([-107,-92.5,25,36.5], crs = ccrs.PlateCarree())

    ax21 = fig.add_axes([0.7, 2.5, 0.5, 0.5],projection = proj)
    ax21.add_feature(cfeat.BORDERS.with_scale('50m'), linewidth=0.8, zorder=1)
    ax21.add_feature(cfeat.COASTLINE.with_scale('50m'), linewidth=0.6, zorder=1)
    gl = ax21.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.2, color='k', alpha=0.5, linestyle='--')
    gl.ylabels_right = False
    gl.xlabels_top = False
    gl.xlines = False
    gl.ylines = False
    world = shpreader.Reader(shp_path).geometries()
    ax21.add_geometries(world, ccrs.PlateCarree(),facecolor='none', edgecolor='black',zorder = 2)
    c21 = ax21.contourf(nx, ny, pdsi[6,::],levels=levels,cmap = cmap, transform = ccrs.PlateCarree(),zorder=1)
    ax21.set_extent([-107,-92.5,25,36.5], crs = ccrs.PlateCarree())

    ax31 = fig.add_axes([1.3, 2.5, 0.5, 0.5],projection = proj)
    ax31.add_feature(cfeat.BORDERS.with_scale('50m'), linewidth=0.8, zorder=1)
    ax31.add_feature(cfeat.COASTLINE.with_scale('50m'), linewidth=0.6, zorder=1)
    gl = ax31.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.2, color='k', alpha=0.5, linestyle='--')
    gl.ylabels_right = False
    gl.xlabels_top = False
    gl.xlines = False
    gl.ylines = False
    world = shpreader.Reader(shp_path).geometries()
    ax31.add_geometries(world, ccrs.PlateCarree(),facecolor='none', edgecolor='black',zorder = 2)
    c31 = ax31.contourf(nx, ny, pdsi[7,::],levels=levels,cmap = cmap, transform = ccrs.PlateCarree())
    ax31.set_extent([-107,-92.5,25,36.5], crs = ccrs.PlateCarree())

    ax41 = fig.add_axes([1.9, 2.5, 0.5, 0.5],projection = proj)
    ax41.add_feature(cfeat.BORDERS.with_scale('50m'), linewidth=0.8, zorder=1)
    ax41.add_feature(cfeat.COASTLINE.with_scale('50m'), linewidth=0.6, zorder=1)
    gl = ax41.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.2, color='k', alpha=0.5, linestyle='--')
    gl.ylabels_right = False
    gl.xlabels_top = False
    gl.xlines = False
    gl.ylines = False
    world = shpreader.Reader(shp_path).geometries()
    ax41.add_geometries(world, ccrs.PlateCarree(),facecolor='none', edgecolor='black',zorder = 2)
    c41 = ax41.contourf(nx, ny, pdsi[8,::],levels=levels,cmap = cmap, transform = ccrs.PlateCarree())
    ax41.set_extent([-107,-92.5,25,36.5], crs = ccrs.PlateCarree())

    ax51 = fig.add_axes([2.5, 2.5, 0.5, 0.5],projection = proj)
    ax51.add_feature(cfeat.BORDERS.with_scale('50m'), linewidth=0.8, zorder=1)
    ax51.add_feature(cfeat.COASTLINE.with_scale('50m'), linewidth=0.6, zorder=1)
    gl = ax51.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.2, color='k', alpha=0.5, linestyle='--')
    gl.ylabels_right = False
    gl.xlabels_top = False
    gl.xlines = False
    gl.ylines = False
    world = shpreader.Reader(shp_path).geometries()
    ax51.add_geometries(world, ccrs.PlateCarree(),facecolor='none', edgecolor='black',zorder = 2)
    c51 = ax51.contourf(nx, ny, pdsi[9,::],levels=levels,cmap = cmap, transform = ccrs.PlateCarree())
    ax51.set_extent([-107,-92.5,25,36.5], crs = ccrs.PlateCarree())

    ax12 = fig.add_axes([0.1, 2, 0.5, 0.5],projection = proj)
    ax12.add_feature(cfeat.BORDERS.with_scale('50m'), linewidth=0.8, zorder=1)
    ax12.add_feature(cfeat.COASTLINE.with_scale('50m'), linewidth=0.6, zorder=1)
    gl = ax12.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.2, color='k', alpha=0.5, linestyle='--')
    gl.ylabels_right = False
    gl.xlabels_top = False
    gl.xlines = False
    gl.ylines = False
    world = shpreader.Reader(shp_path).geometries()
    ax12.add_geometries(world, ccrs.PlateCarree(),facecolor='none', edgecolor='black',zorder = 2)
    c12 = ax12.contourf(nx, ny, pdsi[10,::],levels=levels,cmap = cmap, transform = ccrs.PlateCarree())
    ax12.set_extent([-107,-92.5,25,36.5], crs = ccrs.PlateCarree())

    ax22 = fig.add_axes([0.7, 2, 0.5, 0.5],projection = proj)
    ax22.add_feature(cfeat.BORDERS.with_scale('50m'), linewidth=0.8, zorder=1)
    ax22.add_feature(cfeat.COASTLINE.with_scale('50m'), linewidth=0.6, zorder=1)
    gl = ax22.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.2, color='k', alpha=0.5, linestyle='--')
    gl.ylabels_right = False
    gl.xlabels_top = False
    gl.xlines = False
    gl.ylines = False
    world = shpreader.Reader(shp_path).geometries()
    ax22.add_geometries(world, ccrs.PlateCarree(),facecolor='none', edgecolor='black',zorder = 2)
    c22 = ax22.contourf(nx, ny, pdsi[11,::],levels=levels,cmap = cmap, transform = ccrs.PlateCarree(),zorder=1)
    ax22.set_extent([-107,-92.5,25,36.5], crs = ccrs.PlateCarree())

    ax32 = fig.add_axes([1.3, 2, 0.5, 0.5],projection = proj)
    ax32.add_feature(cfeat.BORDERS.with_scale('50m'), linewidth=0.8, zorder=1)
    ax32.add_feature(cfeat.COASTLINE.with_scale('50m'), linewidth=0.6, zorder=1)
    gl = ax32.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.2, color='k', alpha=0.5, linestyle='--')
    gl.ylabels_right = False
    gl.xlabels_top = False
    gl.xlines = False
    gl.ylines = False
    world = shpreader.Reader(shp_path).geometries()
    ax32.add_geometries(world, ccrs.PlateCarree(),facecolor='none', edgecolor='black',zorder = 2)
    c32 = ax32.contourf(nx, ny, pdsi[12,::],levels=levels,cmap = cmap, transform = ccrs.PlateCarree())
    ax32.set_extent([-107,-92.5,25,36.5], crs = ccrs.PlateCarree())

    ax42 = fig.add_axes([1.9, 2, 0.5, 0.5],projection = proj)
    ax42.add_feature(cfeat.BORDERS.with_scale('50m'), linewidth=0.8, zorder=1)
    ax42.add_feature(cfeat.COASTLINE.with_scale('50m'), linewidth=0.6, zorder=1)
    gl = ax42.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.2, color='k', alpha=0.5, linestyle='--')
    gl.ylabels_right = False
    gl.xlabels_top = False
    gl.xlines = False
    gl.ylines = False
    world = shpreader.Reader(shp_path).geometries()
    ax42.add_geometries(world, ccrs.PlateCarree(),facecolor='none', edgecolor='black',zorder = 2)
    c42 = ax42.contourf(nx, ny, pdsi[13,::],levels=levels,cmap = cmap, transform = ccrs.PlateCarree())
    ax42.set_extent([-107,-92.5,25,36.5], crs = ccrs.PlateCarree())

    ax52 = fig.add_axes([2.5, 2, 0.5, 0.5],projection = proj)
    ax52.add_feature(cfeat.BORDERS.with_scale('50m'), linewidth=0.8, zorder=1)
    ax52.add_feature(cfeat.COASTLINE.with_scale('50m'), linewidth=0.6, zorder=1)
    gl = ax52.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.2, color='k', alpha=0.5, linestyle='--')
    gl.ylabels_right = False
    gl.xlabels_top = False
    gl.xlines = False
    gl.ylines = False
    world = shpreader.Reader(shp_path).geometries()
    ax52.add_geometries(world, ccrs.PlateCarree(),facecolor='none', edgecolor='black',zorder = 2)
    c52 = ax52.contourf(nx, ny, pdsi[14,::],levels=levels,cmap = cmap, transform = ccrs.PlateCarree())
    ax52.set_extent([-107,-92.5,25,36.5], crs = ccrs.PlateCarree())

    ax13 = fig.add_axes([0.1, 1.5, 0.5, 0.5],projection = proj)
    ax13.add_feature(cfeat.BORDERS.with_scale('50m'), linewidth=0.8, zorder=1)
    ax13.add_feature(cfeat.COASTLINE.with_scale('50m'), linewidth=0.6, zorder=1)
    gl = ax13.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.2, color='k', alpha=0.5, linestyle='--')
    gl.ylabels_right = False
    gl.xlabels_top = False
    gl.xlines = False
    gl.ylines = False
    world = shpreader.Reader(shp_path).geometries()
    ax13.add_geometries(world, ccrs.PlateCarree(),facecolor='none', edgecolor='black',zorder = 2)
    c13 = ax13.contourf(nx, ny, pdsi[15,::],levels=levels,cmap = cmap, transform = ccrs.PlateCarree())
    ax13.set_extent([-107,-92.5,25,36.5], crs = ccrs.PlateCarree())

    ax23 = fig.add_axes([0.7, 1.5, 0.5, 0.5],projection = proj)
    ax23.add_feature(cfeat.BORDERS.with_scale('50m'), linewidth=0.8, zorder=1)
    ax23.add_feature(cfeat.COASTLINE.with_scale('50m'), linewidth=0.6, zorder=1)
    gl = ax23.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.2, color='k', alpha=0.5, linestyle='--')
    gl.ylabels_right = False
    gl.xlabels_top = False
    gl.xlines = False
    gl.ylines = False
    world = shpreader.Reader(shp_path).geometries()
    ax23.add_geometries(world, ccrs.PlateCarree(),facecolor='none', edgecolor='black',zorder = 2)
    c23 = ax23.contourf(nx, ny, pdsi[16,::],levels=levels,cmap = cmap, transform = ccrs.PlateCarree(),zorder=1)
    ax23.set_extent([-107,-92.5,25,36.5], crs = ccrs.PlateCarree())

    ax33 = fig.add_axes([1.3, 1.5, 0.5, 0.5],projection = proj)
    ax33.add_feature(cfeat.BORDERS.with_scale('50m'), linewidth=0.8, zorder=1)
    ax33.add_feature(cfeat.COASTLINE.with_scale('50m'), linewidth=0.6, zorder=1)
    gl = ax33.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.2, color='k', alpha=0.5, linestyle='--')
    gl.ylabels_right = False
    gl.xlabels_top = False
    gl.xlines = False
    gl.ylines = False
    world = shpreader.Reader(shp_path).geometries()
    ax33.add_geometries(world, ccrs.PlateCarree(),facecolor='none', edgecolor='black',zorder = 2)
    c33 = ax33.contourf(nx, ny, pdsi[17,::],levels=levels,cmap = cmap, transform = ccrs.PlateCarree())
    ax33.set_extent([-107,-92.5,25,36.5], crs = ccrs.PlateCarree())

    ax43 = fig.add_axes([1.9, 1.5, 0.5, 0.5],projection = proj)
    ax43.add_feature(cfeat.BORDERS.with_scale('50m'), linewidth=0.8, zorder=1)
    ax43.add_feature(cfeat.COASTLINE.with_scale('50m'), linewidth=0.6, zorder=1)
    gl = ax43.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.2, color='k', alpha=0.5, linestyle='--')
    gl.ylabels_right = False
    gl.xlabels_top = False
    gl.xlines = False
    gl.ylines = False
    world = shpreader.Reader(shp_path).geometries()
    ax43.add_geometries(world, ccrs.PlateCarree(),facecolor='none', edgecolor='black',zorder = 2)
    c43 = ax43.contourf(nx, ny, pdsi[18,::],levels=levels,cmap = cmap, transform = ccrs.PlateCarree())
    ax43.set_extent([-107,-92.5,25,36.5], crs = ccrs.PlateCarree())

    ax53 = fig.add_axes([2.5, 1.5, 0.5, 0.5],projection = proj)
    ax53.add_feature(cfeat.BORDERS.with_scale('50m'), linewidth=0.8, zorder=1)
    ax53.add_feature(cfeat.COASTLINE.with_scale('50m'), linewidth=0.6, zorder=1)
    gl = ax53.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.2, color='k', alpha=0.5, linestyle='--')
    gl.ylabels_right = False
    gl.xlabels_top = False
    gl.xlines = False
    gl.ylines = False
    world = shpreader.Reader(shp_path).geometries()
    ax53.add_geometries(world, ccrs.PlateCarree(),facecolor='none', edgecolor='black',zorder = 2)
    c53 = ax53.contourf(nx, ny, pdsi[19,::],levels=levels,cmap = cmap, transform = ccrs.PlateCarree())
    ax53.set_extent([-107,-92.5,25,36.5], crs = ccrs.PlateCarree())

    ax14 = fig.add_axes([0.1, 1, 0.5, 0.5],projection = proj)
    ax14.add_feature(cfeat.BORDERS.with_scale('50m'), linewidth=0.8, zorder=1)
    ax14.add_feature(cfeat.COASTLINE.with_scale('50m'), linewidth=0.6, zorder=1)
    gl = ax14.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.2, color='k', alpha=0.5, linestyle='--')
    gl.ylabels_right = False
    gl.xlabels_top = False
    gl.xlines = False
    gl.ylines = False
    world = shpreader.Reader(shp_path).geometries()
    ax14.add_geometries(world, ccrs.PlateCarree(),facecolor='none', edgecolor='black',zorder = 2)
    c14 = ax14.contourf(nx, ny, pdsi[20,::],levels=levels,cmap = cmap, transform = ccrs.PlateCarree())
    ax14.set_extent([-107,-92.5,25,36.5], crs = ccrs.PlateCarree())

    ax24 = fig.add_axes([0.7, 1, 0.5, 0.5],projection = proj)
    ax24.add_feature(cfeat.BORDERS.with_scale('50m'), linewidth=0.8, zorder=1)
    ax24.add_feature(cfeat.COASTLINE.with_scale('50m'), linewidth=0.6, zorder=1)
    gl = ax24.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.2, color='k', alpha=0.5, linestyle='--')
    gl.ylabels_right = False
    gl.xlabels_top = False
    gl.xlines = False
    gl.ylines = False
    world = shpreader.Reader(shp_path).geometries()
    ax24.add_geometries(world, ccrs.PlateCarree(),facecolor='none', edgecolor='black',zorder = 2)
    c24 = ax24.contourf(nx, ny, pdsi[21,::],levels=levels,cmap = cmap, transform = ccrs.PlateCarree(),zorder=1)
    ax24.set_extent([-107,-92.5,25,36.5], crs = ccrs.PlateCarree())

    ax34 = fig.add_axes([1.3, 1, 0.5, 0.5],projection = proj)
    ax34.add_feature(cfeat.BORDERS.with_scale('50m'), linewidth=0.8, zorder=1)
    ax34.add_feature(cfeat.COASTLINE.with_scale('50m'), linewidth=0.6, zorder=1)
    gl = ax34.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.2, color='k', alpha=0.5, linestyle='--')
    gl.ylabels_right = False
    gl.xlabels_top = False
    gl.xlines = False
    gl.ylines = False
    world = shpreader.Reader(shp_path).geometries()
    ax34.add_geometries(world, ccrs.PlateCarree(),facecolor='none', edgecolor='black',zorder = 2)
    c34 = ax34.contourf(nx, ny, pdsi[22,::],levels=levels,cmap = cmap, transform = ccrs.PlateCarree())
    ax34.set_extent([-107,-92.5,25,36.5], crs = ccrs.PlateCarree())

    ax44 = fig.add_axes([1.9, 1, 0.5, 0.5],projection = proj)
    ax44.add_feature(cfeat.BORDERS.with_scale('50m'), linewidth=0.8, zorder=1)
    ax44.add_feature(cfeat.COASTLINE.with_scale('50m'), linewidth=0.6, zorder=1)
    gl = ax44.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.2, color='k', alpha=0.5, linestyle='--')
    gl.ylabels_right = False
    gl.xlabels_top = False
    gl.xlines = False
    gl.ylines = False
    world = shpreader.Reader(shp_path).geometries()
    ax44.add_geometries(world, ccrs.PlateCarree(),facecolor='none', edgecolor='black',zorder = 2)
    c44 = ax44.contourf(nx, ny, pdsi[23,::],levels=levels,cmap = cmap, transform = ccrs.PlateCarree())
    ax44.set_extent([-107,-92.5,25,36.5], crs = ccrs.PlateCarree())

    # ax54 = fig.add_axes([2.5, 1, 0.5, 0.5],projection = proj)
    # ax54.add_feature(cfeat.BORDERS.with_scale('50m'), linewidth=0.8, zorder=1)
    # ax54.add_feature(cfeat.COASTLINE.with_scale('50m'), linewidth=0.6, zorder=1)
    # gl = ax54.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.2, color='k', alpha=0.5, linestyle='--')
    # gl.ylabels_right = False
    # gl.xlabels_top = False
    # gl.xlines = False
    # gl.ylines = False
    # world = shpreader.Reader(shp_path).geometries()
    # ax54.add_geometries(world, ccrs.PlateCarree(),facecolor='none', edgecolor='black',zorder = 2)
    # c54 = ax54.contourf(nx, ny, pdsi[24,::],levels=levels,cmap = cmaps.MPL_OrRd_r, transform = ccrs.PlateCarree())
    # ax54.set_extent([-107,-92.5,25,36.5], crs = ccrs.PlateCarree())



    # #添加南海，实际上就是新建一个子图覆盖在之前子图的右下角
    # ax7 = fig.add_axes([0.5, 1.01, 0.12, 0.1], projection=ccrs.PlateCarree())
    # ax7.set_extent([107, 122,3,25])
    # # ax7.add_geometries(Reader(r'E:\Research_life\毕设分区\矢量图层-20210720T085947Z-001\矢量图层\国界与省界\bou2_4l.shp').geometries(),ccrs.PlateCarree(),facecolor='none',edgecolor='k',linewidth=0.8)

    # 添加色标，position定义色标位置，c1指定从c1填色图层取色，由于C3,C1的levles相同，所以色标一致，orientation设置色标为水平还是垂直，format设置色标标签格式
    position1 = fig.add_axes([0.2, 0.9, 2.4, 0.05])
    cbar=fig.colorbar(c1,cax=position1,orientation='horizontal',format='%.2f')
    cbar.set_label('$\mathrm{drought_pro}$',fontsize = 54)
    return

# if __name__ == '__main__':
#     Lon = Lon
#     Lat = Lat
#     # slope = np.zeros((62,42))
#     # Multi_spatial_graph_PDSI(pdsi.lon.values,pdsi.lat.values,pdsi,2010,2011)
#     Multi_spatial_graph_PDSI(drought_pro_nc.lon.values,drought_pro_nc,drought_pro_nc,2010,2011)



In [ ]:
# spei = xr.open_dataset("/mnt/e/Research_life/DATA/SPEI/spei03.nc").spei
# Multi_spatial_graph_PDSI(spei.lon.values,spei.lat.values,spei,2010,2011)
# Multi_spatial_graph_drought_pro(drought_pro_nc.drought_por.lon.values,drought_pro_nc.drought_por.lat.values,drought_pro_nc.drought_por,2010,2011)

drought_pro_modify = xr.open_dataset('/mnt/e/Research_life/DATA/RF_model/drought_pro_modify.nc')
Multi_spatial_graph_drought_pro(drought_pro_modify.drought_por.lon.values,drought_pro_modify.drought_por.lat.values,drought_pro_modify.drought_por,2010,2011)


# spei

In [ ]:
# Pre[0,::].plot()
ds1 = xr.open_dataset("/mnt/e/Research_life/DATA/GLEAM/data/v3.6a/monthly/Ep_1980-2021_GLEAM_v3.6a_MO.nc").Ep
# ds.loc['1982':,36:25,-110:-94][0,::].plot()
# DS = ds.interp(
#     lat = Lat,
#     lon = Lon,
#     method_non_numeric='nearest',
#     kwargs={
#         "fill_value":"extrapolate"
#         }
#     )
# ds1.loc['1982':,36:25,-110:-94][0,::].plot()
# DS.loc['1982':,25:36,-110:-94][0,::].plot()
# DS

# ds =xr.open_dataset("/mnt/e/Research_life/DATA/CRU TS v4.02/cru_ts4.02.1901.2017.pet.dat.nc/cru_ts4.02.1901.2017.pet.dat.nc")


ds.pet.loc['1982':,25:36,-110:-94][0,::].plot()

In [ ]:
Lat = np.arange(19.5,50.5,0.5)
Lon=np.arange(-110.5,-89.5,0.5)
Multi_spatial_graph(Lon,Lat,drought_pro)

In [ ]:
# identify_staionpoint_isin_whichgird(Lon,Lat,loc,'Potter')
# Lat==35.5
# np.where(Lon==-102.5)
a = drought_pro[:,32,16]
# ## save drought_pro dataset
# import scipy.io as io
# io.savemat('/mnt/e/Research_life/DATA/RF_model/drought_pro.mat', {'name': drought_pro})
pd.DataFrame(a).to_csv('/mnt/e/Research_life/DATA/RF_model/a.csv')

# compare drought_por with SPEI and other aridity index

In [ ]:
## spei
spei = xr.open_dataset("/mnt/e/Research_life/DATA/SPEI/spei04.nc").spei
SPEI = spei.interp(
    lat = Lat,
    lon = Lon,
    method_non_numeric='nearest',
    kwargs={
        "fill_value":"extrapolate"
        }
    )

##sc-pdsi
pdsi = xr.open_dataset("/mnt/e/Research_life/DATA/sc-pdsi/pdsi.mon.mean.selfcalibrated.nc").pdsi
PDSI = pdsi.interp(
    lat = Lat,
    lon = Lon,
    method_non_numeric='nearest',
    kwargs={
        "fill_value":"extrapolate"
        }
    )

drought_pro_nc = xr.open_dataset('/mnt/e/Research_life/DATA/RF_model/drought_pro.nc')


In [ ]:
start_yr = 2011
end_yr = 2011
for mon_id in range(12):
# mon_id = 5  #the 6th month (mon_id range from 0-11)
    # Multi_spatial_graph_SPEI(Lon,Lat,SPEI.loc[str(start_yr):str(end_yr),::][mon_id,::].values)
    Multi_spatial_graph(Lon,Lat,drought_pro[(end_yr-1982)*12+mon_id,::])
    # Multi_spatial_graph_PDSI(pdsi.lon.values,pdsi.lat.values,pdsi.loc[str(start_yr):str(end_yr),::][mon_id,::].values)



In [ ]:
np.nanmin(r)

### corr analysis

In [ ]:

import xarray as xr
from scipy.stats import pearsonr
import numpy as np
import cartopy.crs as ccrs 
import cartopy.feature as cfeat 
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER 
import matplotlib.pyplot as plt
import cmaps
import pandas as pd

## corr analysis
r = np.zeros((len(grace.lat),len(grace.lon)))
p = np.zeros((len(grace.lat),len(grace.lon)))
a = SPEI.loc[str(1982):str(2015),::]
# a = PDSI.loc[str(1982):str(2015),::]
## 相关性分析，网格形状
for i in range(len(grace.lat)):
    for j in range(len(grace.lon)):
        try:
            data = pd.DataFrame({'SIF':drought_pro[:,i,j],'VPD':a.values[:,i,j]})
            data = data.dropna()
            r[i,j], p[i,j] = pearsonr(data['SIF'],data['VPD'])
        except:
            r[i,j] = np.nan
            p[i,j] = np.nan


# Multi_spatial_graph(Lon,Lat,r)


def Multi_spatial_one_graph(lon,lat,slope1):
    from matplotlib import rcParams
    import numpy as np
    import cartopy.crs as ccrs
    import matplotlib.pyplot as plt
    import cmaps
    import cartopy.io.shapereader as shpreader
    import matplotlib as mpl
    from cartopy.io.shapereader import Reader, natural_earth
    import cartopy.feature as cfeat
    from matplotlib.colors import ListedColormap 

    config = {
        "font.family": 'Times New Roman', # times new roman字体
        "font.size": 18, # 相当于小四大小
        "font.serif": ['SimSun'], # 宋体
        "mathtext.fontset": 'stix', # matplotlib渲染数学字体时使用的字体，和Times New Roman差别不大
        'axes.unicode_minus': False # 处理负号，即-号
    }
    rcParams.update(config)
    plt.rcParams['axes.unicode_minus']=False #用来正常显示负号
    plt.rcParams['xtick.direction'] = 'in'#将x周的刻度线方向设置向内
    plt.rcParams['ytick.direction'] = 'in'#将y轴的刻度方向设置向内
    shp_path = '/mnt/e/Research_life/ArcGIS/USA/USA-texas/Texas_County_Boundaries_Detailed/County.shp'
    nx, ny = np.meshgrid(lon, lat)
    fig = plt.figure(figsize=(20,20), dpi = 300)
    proj = ccrs.PlateCarree()
    
    # 使图一colorbar映射在同一水平
    # norm1 = mpl.colors.Normalize(vmin=-abs(np.nanmax(slope1)), vmax=abs(np.nanmax(slope1)))
    # norm1 = mpl.colors.BoundaryNorm(np.arange(0.2,1.1,0.1),cmaps.MPL_OrRd.N)
    # newcolors = cmaps.MPL_BrBG(np.linspace(0,1,5))
    # newcmap=ListedColormap(newcolors[::1]) 


    ## graph one
    ax1 = fig.add_axes([0.1, 0.9, 0.5, 0.5],projection = proj)
    ax1.add_feature(cfeat.BORDERS.with_scale('50m'), linewidth=0.8, zorder=1)
    ax1.add_feature(cfeat.COASTLINE.with_scale('50m'), linewidth=0.6, zorder=1)
    # ax1.set_title('(a)',loc='left')
    gl = ax1.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.2, color='k', alpha=0.5, linestyle='--')
    gl.ylabels_right = False
    gl.xlabels_top = False
    gl.xlines = False
    gl.ylines = False
    world = shpreader.Reader(shp_path).geometries()
    ax1.add_geometries(world, ccrs.PlateCarree(),facecolor='none', edgecolor='black',zorder = 2)
    # 颜色映射，以0为分界线
    levels = np.arange(-1,0.1,0.1)
    c1 = ax1.contourf(nx, ny, slope1,levels=levels,cmap = cmaps.MPL_Blues_r, transform = ccrs.PlateCarree())
    # c1 = ax1.contourf(nx, ny, slope1,levels=levels,cmap = newcmap, transform = ccrs.PlateCarree())

    c1.set_clim(vmin=-1, vmax=-0.3)
    # c1.set_clim(vmin=-0.2, vmax=0.4)
    ax1.set_extent([-107,-92.5,25,36.5], crs = ccrs.PlateCarree())
    
    # 添加色标，position定义色标位置，c1指定从c1填色图层取色，由于C3,C1的levles相同，所以色标一致，orientation设置色标为水平还是垂直，format设置色标标签格式
    position1 = fig.add_axes([0.1, 0.86, 0.5, 0.02])
    cbar=fig.colorbar(c1,cax=position1,orientation='horizontal',format='%.2f')
    cbar.set_label('$\mathrm{dorught_pro}$',fontsize = 24)
    return

if __name__ == '__main__':
    Lon = Lon
    Lat = Lat
    # slope = np.zeros((62,42))
    Multi_spatial_one_graph(Lon,Lat,r)
    # Multi_spatial_graph(Lon,Lat,slope)